In [2]:
import requests
import csv
import os
from datetime import date, timedelta

# Ngày hôm qua
yesterday = date.today() - timedelta(days=1)
# Lấy 30 ngày trước
days_back = 30
start_date = yesterday - timedelta(days=days_back-1)  # -1 vì đã tính cả ngày hôm qua
end_date = yesterday

# Tạo thư mục lưu dữ liệu
os.makedirs("weather_archive", exist_ok=True)
csv_file = f"weather_archive/{start_date.isoformat()}_{end_date.isoformat()}.csv"
header = ["Province", "Date", "Temp_Max", "Temp_Min", "Weather_Code"]

# Danh sách tỉnh và toạ độ
provinces = [
    {"name": "An Giang", "lat": 10.5, "lon": 105.16},
    {"name": "Bà Rịa - Vũng Tàu", "lat": 10.35, "lon": 107.08},
    {"name": "Bạc Liêu", "lat": 9.29, "lon": 105.72},
    {"name": "Bắc Giang", "lat": 21.28, "lon": 106.2},
    {"name": "Bắc Kạn", "lat": 22.14, "lon": 105.84},
    {"name": "Bắc Ninh", "lat": 21.18, "lon": 106.05},
    {"name": "Bến Tre", "lat": 10.23, "lon": 106.38},
    {"name": "Bình Định", "lat": 13.77, "lon": 109.22},
    {"name": "Bình Dương", "lat": 11.18, "lon": 106.68},
    {"name": "Bình Phước", "lat": 11.75, "lon": 106.72},
    {"name": "Bình Thuận", "lat": 10.93, "lon": 108.1},
    {"name": "Cà Mau", "lat": 9.18, "lon": 105.15},
    {"name": "Cần Thơ", "lat": 10.03, "lon": 105.78},
    {"name": "Cao Bằng", "lat": 22.66, "lon": 106.25},
    {"name": "Đà Nẵng", "lat": 16.07, "lon": 108.22},
    {"name": "Đắk Lắk", "lat": 12.71, "lon": 108.23},
    {"name": "Đắk Nông", "lat": 12.0, "lon": 107.7},
    {"name": "Điện Biên", "lat": 21.38, "lon": 103.02},
    {"name": "Đồng Nai", "lat": 10.94, "lon": 106.82},
    {"name": "Đồng Tháp", "lat": 10.49, "lon": 105.69},
    {"name": "Gia Lai", "lat": 13.98, "lon": 108.0},
    {"name": "Hà Giang", "lat": 22.78, "lon": 104.98},
    {"name": "Hà Nam", "lat": 20.55, "lon": 105.92},
    {"name": "Hà Nội", "lat": 21.03, "lon": 105.85},
    {"name": "Hà Tĩnh", "lat": 18.34, "lon": 105.91},
    {"name": "Hải Dương", "lat": 20.94, "lon": 106.33},
    {"name": "Hải Phòng", "lat": 20.86, "lon": 106.68},
    {"name": "Hậu Giang", "lat": 9.78, "lon": 105.64},
    {"name": "Hòa Bình", "lat": 20.82, "lon": 105.33},
    {"name": "Hưng Yên", "lat": 20.65, "lon": 106.05},
    {"name": "Khánh Hòa", "lat": 12.24, "lon": 109.19},
    {"name": "Kiên Giang", "lat": 10.02, "lon": 105.08},
    {"name": "Kon Tum", "lat": 14.35, "lon": 108.0},
    {"name": "Lai Châu", "lat": 22.4, "lon": 103.45},
    {"name": "Lâm Đồng", "lat": 11.94, "lon": 108.44},
    {"name": "Lạng Sơn", "lat": 21.85, "lon": 106.76},
    {"name": "Lào Cai", "lat": 22.48, "lon": 103.97},
    {"name": "Long An", "lat": 10.61, "lon": 106.67},
    {"name": "Nam Định", "lat": 20.43, "lon": 106.18},
    {"name": "Nghệ An", "lat": 19.23, "lon": 104.92},
    {"name": "Ninh Bình", "lat": 20.25, "lon": 105.97},
    {"name": "Ninh Thuận", "lat": 11.67, "lon": 108.86},
    {"name": "Phú Thọ", "lat": 21.32, "lon": 105.4},
    {"name": "Phú Yên", "lat": 13.1, "lon": 109.3},
    {"name": "Quảng Bình", "lat": 17.47, "lon": 106.62},
    {"name": "Quảng Nam", "lat": 15.54, "lon": 108.02},
    {"name": "Quảng Ngãi", "lat": 15.12, "lon": 108.8},
    {"name": "Quảng Ninh", "lat": 21.01, "lon": 107.29},
    {"name": "Quảng Trị", "lat": 16.81, "lon": 107.1},
    {"name": "Sóc Trăng", "lat": 9.6, "lon": 105.98},
    {"name": "Sơn La", "lat": 21.33, "lon": 103.91},
    {"name": "Tây Ninh", "lat": 11.31, "lon": 106.1},
    {"name": "Thái Bình", "lat": 20.45, "lon": 106.34},
    {"name": "Thái Nguyên", "lat": 21.59, "lon": 105.84},
    {"name": "Thanh Hóa", "lat": 19.8, "lon": 105.76},
    {"name": "Thừa Thiên Huế", "lat": 16.46, "lon": 107.59},
    {"name": "Tiền Giang", "lat": 10.36, "lon": 106.36},
    {"name": "TP. Hồ Chí Minh", "lat": 10.82, "lon": 106.63},
    {"name": "Trà Vinh", "lat": 9.94, "lon": 106.34},
    {"name": "Tuyên Quang", "lat": 21.82, "lon": 105.21},
    {"name": "Vĩnh Long", "lat": 10.25, "lon": 105.97},
    {"name": "Vĩnh Phúc", "lat": 21.3, "lon": 105.6},
    {"name": "Yên Bái", "lat": 21.71, "lon": 104.91}
]

print(f"Đang cào dữ liệu từ {start_date.isoformat()} đến {end_date.isoformat()}")

with open(csv_file, mode="w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(header)

    total_provinces = len(provinces)
    for idx, province in enumerate(provinces, 1):
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": province["lat"],
            "longitude": province["lon"],
            "start_date": start_date.isoformat(),
            "end_date": end_date.isoformat(),
            "daily": "temperature_2m_max,temperature_2m_min,weathercode",
            "timezone": "Asia/Bangkok"
        }

        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                daily_data = response.json().get("daily", {})
                dates = daily_data.get("time", [])
                temp_maxs = daily_data.get("temperature_2m_max", [])
                temp_mins = daily_data.get("temperature_2m_min", [])
                weathercodes = daily_data.get("weathercode", [])

                for i in range(len(dates)):
                    writer.writerow([
                        province["name"],
                        dates[i],
                        temp_maxs[i],
                        temp_mins[i],
                        weathercodes[i]
                    ])
                print(f"✅ [{idx}/{total_provinces}] {province['name']}: xong")
            else:
                print(f"❌ [{idx}/{total_provinces}] {province['name']}: lỗi API (HTTP {response.status_code})")
                print(f"   Chi tiết: {response.text}")
        except Exception as e:
            print(f"❌ [{idx}/{total_provinces}] {province['name']}: lỗi kết nối - {str(e)}")
            # Thêm thời gian chờ nếu bị lỗi kết nối
            import time
            time.sleep(2)

print(f"\n📄 Đã lưu vào file: {csv_file}")
print(f"📊 Dữ liệu bao gồm {days_back} ngày, từ {start_date.isoformat()} đến {end_date.isoformat()}")

Đang cào dữ liệu từ 2025-04-06 đến 2025-05-05
✅ [1/63] An Giang: xong
✅ [2/63] Bà Rịa - Vũng Tàu: xong
✅ [3/63] Bạc Liêu: xong
✅ [4/63] Bắc Giang: xong
✅ [5/63] Bắc Kạn: xong
✅ [6/63] Bắc Ninh: xong
✅ [7/63] Bến Tre: xong
✅ [8/63] Bình Định: xong
✅ [9/63] Bình Dương: xong
✅ [10/63] Bình Phước: xong
✅ [11/63] Bình Thuận: xong
✅ [12/63] Cà Mau: xong
✅ [13/63] Cần Thơ: xong
✅ [14/63] Cao Bằng: xong
✅ [15/63] Đà Nẵng: xong
✅ [16/63] Đắk Lắk: xong
✅ [17/63] Đắk Nông: xong
✅ [18/63] Điện Biên: xong
✅ [19/63] Đồng Nai: xong
✅ [20/63] Đồng Tháp: xong
✅ [21/63] Gia Lai: xong
✅ [22/63] Hà Giang: xong
✅ [23/63] Hà Nam: xong
✅ [24/63] Hà Nội: xong
✅ [25/63] Hà Tĩnh: xong
✅ [26/63] Hải Dương: xong
✅ [27/63] Hải Phòng: xong
✅ [28/63] Hậu Giang: xong
✅ [29/63] Hòa Bình: xong
✅ [30/63] Hưng Yên: xong
✅ [31/63] Khánh Hòa: xong
✅ [32/63] Kiên Giang: xong
✅ [33/63] Kon Tum: xong
✅ [34/63] Lai Châu: xong
✅ [35/63] Lâm Đồng: xong
✅ [36/63] Lạng Sơn: xong
✅ [37/63] Lào Cai: xong
✅ [38/63] Long An: xong
✅ [3

In [3]:
import pandas as pd
df = pd.read_csv('weather_archive/2025-04-05_2025-05-05.csv')

In [4]:
df.head(5)

,Province,Date,Temp_Max,Temp_Min,Weather_Code
0,An Giang,2025-04-06,33.0,23.6,3.0
1,An Giang,2025-04-07,33.8,24.2,3.0
2,An Giang,2025-04-08,33.8,23.8,3.0
3,An Giang,2025-04-09,33.8,24.3,63.0
4,An Giang,2025-04-10,30.9,24.4,61.0
